In [23]:
import cobra.test
import os
from os.path import join
from cobra.io import read_sbml_model
model = cobra.io.load_json_model(join("RehMBEL1391_sbml_L3V1.json"))
from cobra import Model, Reaction, Metabolite

# Adding D-glucose to the medium

The model we are using has D-fructorse as carbon source.

## Standard medium content of the model

In [24]:
model.medium   #EX_fru_e is D-frucose

{'EX_fe2_e': 10.0,
 'EX_mg2_e': 10.0,
 'EX_pi_e': 100.0,
 'EX_cobalt2_e': 10.0,
 'EX_cl_e': 10.0,
 'EX_k_e': 10.0,
 'EX_fe3_e': 10.0,
 'EX_so4_e': 10.0,
 'EX_fru_e': 3.0,
 'EX_nh4_e': 10.0,
 'EX_na_e': 10.0,
 'EX_o2_e': 18.5,
 'EX_mobd_e': 10.0,
 'EX_h2o_e': 1000.0,
 'EX_h_e': 100.0}

In [25]:
with model:
    medium = model.medium
    medium['EX_fru_e'] = 0
    print(medium)
    model.medium = medium
    solution= model.optimize()
    print('The current theoretical maximum biomass productivity:', solution.fluxes['EX_BIOMASS_c'], 'h')
    print('----------------')
    
    model.objective = model.reactions.EX_pbhb_e
    phb_production=model.optimize().objective_value
    print('The current maximum theoretical productivity for making PHB:', phb_production , 'mmol/gDW*h')
    print('----------------')
    
    #maximum_yield = phb_production / (-1*(model.reactions.EX_fru_e.flux))
    #print('Maximum theoretical yield =', maximum_yield, 'mmol phb/mmol Fructose')
    
    medium = model.medium
    #medium['EX_fru_e'] = 3
    #print(medium)
    model.medium=medium

{'EX_fe2_e': 10.0, 'EX_mg2_e': 10.0, 'EX_pi_e': 100.0, 'EX_cobalt2_e': 10.0, 'EX_cl_e': 10.0, 'EX_k_e': 10.0, 'EX_fe3_e': 10.0, 'EX_so4_e': 10.0, 'EX_fru_e': 0, 'EX_nh4_e': 10.0, 'EX_na_e': 10.0, 'EX_o2_e': 18.5, 'EX_mobd_e': 10.0, 'EX_h2o_e': 1000.0, 'EX_h_e': 100.0}
The current theoretical maximum biomass productivity: 0.0 h
----------------
The current maximum theoretical productivity for making PHB: 0.0 mmol/gDW*h
----------------


In [26]:
#model.reactions.query('pyr', 'name')

We will be trying to include D-glucose in the medium and look at the bimass and production of PHB.

With the current medium the biomass and yeald of phb

In [27]:
with model:
    solution= model.optimize()
    print('The current theoretical maximum biomass productivity:', solution.fluxes['EX_BIOMASS_c'], 'h')
    print('----------------')
    model.objective = model.reactions.EX_pbhb_e
    phb_production=model.optimize().objective_value
    print('The current maximum theoretical productivity for making PHB:', phb_production , 'mmol/gDW*h')
    print('----------------')
    maximum_yield = phb_production / (-1*(model.reactions.EX_fru_e.flux))
    print('Maximum theoretical yield =', maximum_yield, 'mmol phb/mmol fructose') 

The current theoretical maximum biomass productivity: 0.285285866427677 h
----------------
The current maximum theoretical productivity for making PHB: 3.762711864406775 mmol/gDW*h
----------------
Maximum theoretical yield = 1.2542372881355917 mmol phb/mmol fructose


In [34]:
model.metabolites.get_by_id("phb_c")

Metabolite identifier,phb_c
Name,Poly-beta-hydroxybutyrate
Memory address,0x07ff96c9204c0
Formula,None
Compartment,c
In 3 reaction(s),"PHBD, PHAS, PHBt"


### To add D-glucose to the medium we need new reactions to make R.eutropha take up and use D-glucose as a carbon source. 

In [7]:
for reactions in model.metabolites.get_by_id('fru_e').reactions:
    print(reactions)

FRUpts2: fru_e + pep_c --> f6p_c + h_c + pyr_c
FRUpts: fru_e + pep_c --> f1p_c + h_c + pyr_c
EX_fru_e: fru_e <=> 
FRUabc: atp_c + fru_e + h2o_c --> adp_c + fru_c + h_c + pi_c


In [8]:
#EX_glc_D_e = Metabolite(
#    'EX_glc_D_e',
#    formula='C6H12O6',
#    name='D-glucose',
#    compartment='e')

In [9]:
model.add_metabolites([
    Metabolite(
    'glc_e',
    name='D-glucose',
    compartment='e'
    )])

In [10]:
model.add_boundary(model.metabolites.get_by_id("glc_e"), type="exchange")

Reaction identifier,EX_glc_e
Name,D-glucose exchange
Memory address,0x07ff96bc0e9d0
Stoichiometry,glc_e <=> D-glucose <=>
GPR,
Lower bound,-1000.0
Upper bound,1000.0


In [11]:
reaction = Reaction('PEP')

In [12]:
reaction.add_metabolites({
    #model.metabolites.glc_e: -1.0,
    #model.metabolites.pep_c: -1.0,
    #model.metabolites.pyr_c: 1.0,
    #model.metabolites.g6p_c: 1.0
                            })
print(reaction.build_reaction_string())

 --> 


In [13]:
with model:
    medium = model.medium
    medium['EX_glc_e'] = 3
    medium['EX_fru_e'] = 0
    print(medium)
    #model.medium = medium
    solution= model.optimize()
    print('The current theoretical maximum biomass productivity:', solution.fluxes['EX_BIOMASS_c'], 'h')
    print('----------------')
    
    model.objective = model.reactions.EX_pbhb_e
    phb_production=model.optimize().objective_value
    print('The current maximum theoretical productivity for making PHB:', phb_production , 'mmol/gDW*h')
    print('----------------')
    
    maximum_yield = phb_production / (-1*(model.reactions.EX_fru_e.flux))
    print('Maximum theoretical yield =', maximum_yield, 'mmol phb/mmol Fructose')
    
    medium = model.medium
    medium['EX_glc_e'] = 0
    medium['EX_fru_e'] = 3
    print(medium)
    model.medium=medium

{'EX_fe2_e': 10.0, 'EX_mg2_e': 10.0, 'EX_pi_e': 100.0, 'EX_cobalt2_e': 10.0, 'EX_cl_e': 10.0, 'EX_k_e': 10.0, 'EX_fe3_e': 10.0, 'EX_so4_e': 10.0, 'EX_fru_e': 0, 'EX_nh4_e': 10.0, 'EX_na_e': 10.0, 'EX_o2_e': 18.5, 'EX_mobd_e': 10.0, 'EX_h2o_e': 1000.0, 'EX_h_e': 100.0, 'EX_glc_e': 3}
The current theoretical maximum biomass productivity: 0.2852858664276671 h
----------------
The current maximum theoretical productivity for making PHB: 3.7627118644067825 mmol/gDW*h
----------------
Maximum theoretical yield = 1.254237288135594 mmol phb/mmol Fructose
{'EX_fe2_e': 10.0, 'EX_mg2_e': 10.0, 'EX_pi_e': 100.0, 'EX_cobalt2_e': 10.0, 'EX_cl_e': 10.0, 'EX_k_e': 10.0, 'EX_fe3_e': 10.0, 'EX_so4_e': 10.0, 'EX_fru_e': 3, 'EX_nh4_e': 10.0, 'EX_na_e': 10.0, 'EX_o2_e': 18.5, 'EX_mobd_e': 10.0, 'EX_h2o_e': 1000.0, 'EX_h_e': 100.0, 'EX_glc_e': 0}


In [35]:
with model:
    medium = model.medium
    medium['EX_fru_e'] = 0
    medium['EX_succ_e'] = 10
    print(medium)
    model.medium=medium
    
    solution= model.optimize()
    print('The current theoretical maximum biomass productivity:', solution.fluxes['EX_BIOMASS_c'], 'h')
    print('----------------')
    
    model.objective = model.reactions.EX_pbhb_e
    phb_production=model.optimize().objective_value
    print('The current maximum theoretical productivity for making PHB:', phb_production , 'mmol/gDW*h')
    print('----------------')
    
    maximum_yield = phb_production / (-1*(model.reactions.EX_succ_e.flux))
    print('Maximum theoretical yield =', maximum_yield, 'mmol phb/mmol Succinate')
    
    medium = model.medium
    medium['EX_fru_e'] = 3
    medium['EX_succ_e'] = 0
    print(medium)
    model.medium=medium 

{'EX_fe2_e': 10.0, 'EX_mg2_e': 10.0, 'EX_pi_e': 100.0, 'EX_cobalt2_e': 10.0, 'EX_cl_e': 10.0, 'EX_k_e': 10.0, 'EX_fe3_e': 10.0, 'EX_so4_e': 10.0, 'EX_fru_e': 0, 'EX_nh4_e': 10.0, 'EX_na_e': 10.0, 'EX_o2_e': 18.5, 'EX_mobd_e': 10.0, 'EX_h2o_e': 1000.0, 'EX_h_e': 100.0, 'EX_succ_e': 10}
The current theoretical maximum biomass productivity: 0.5343325098873288 h
----------------
The current maximum theoretical productivity for making PHB: 6.454545454545584 mmol/gDW*h
----------------
Maximum theoretical yield = 0.6454545454545584 mmol phb/mmol Succinate
{'EX_fe2_e': 10.0, 'EX_mg2_e': 10.0, 'EX_pi_e': 100.0, 'EX_cobalt2_e': 10.0, 'EX_cl_e': 10.0, 'EX_k_e': 10.0, 'EX_fe3_e': 10.0, 'EX_succ_e': 0, 'EX_so4_e': 10.0, 'EX_nh4_e': 10.0, 'EX_na_e': 10.0, 'EX_o2_e': 18.5, 'EX_mobd_e': 10.0, 'EX_h2o_e': 1000.0, 'EX_h_e': 100.0, 'EX_fru_e': 3}


In [15]:
model.reactions.get_by_id('EX_succ_e')

Reaction identifier,EX_succ_e
Name,EX_succ_e
Memory address,0x07ff96afc8c10
Stoichiometry,succ_e --> Succinate -->
GPR,
Lower bound,0.0
Upper bound,1000.0


In [16]:
for metabolites in model.metabolites.query('succ','name'):
    print(metabolites.name)

Naphthyl-2-methyl-succinic acid
Naphthyl-2-methyl-succinyl-CoA
Naphthyl-2-oxomethyl-succinyl-CoA
1-(5'-Phosphoribosyl)-5-amino-4-(N-succinocarboxamide)-imidazole
Naphthyl-2-hydroxymethyl-succinyl CoA
Naphthyl-2-methylene-succinyl-CoA
N-(L-Arginino)succinate


In [17]:
print(model.reactions.get_by_id('EX_succ_e'))
print(model.metabolites.get_by_id('succ_e').name)

EX_succ_e: succ_e --> 
Succinate


In [18]:
with model:
    medium = model.medium
    medium['EX_o2_e'] = 0
    print(medium)
    model.medium=medium
    solution= model.optimize()
    print('The current theoretical maximum biomass productivity:', solution.fluxes['EX_BIOMASS_c'], 'h')
    print('----------------')
    
    model.objective = model.reactions.EX_pbhb_e
    phb_production=model.optimize().objective_value
    print('The current maximum theoretical productivity for making PHB:', phb_production , 'mmol/gDW*h')
    print('----------------')
    
    maximum_yield = phb_production / (-1*(model.reactions.EX_fru_e.flux))
    print('Maximum theoretical yield =', maximum_yield, 'mmol phb/mmol Fructose')
    
    medium = model.medium
    medium['EX_o2_e'] = 10
    print(medium)
    model.medium=medium 

{'EX_fe2_e': 10.0, 'EX_mg2_e': 10.0, 'EX_pi_e': 100.0, 'EX_cobalt2_e': 10.0, 'EX_cl_e': 10.0, 'EX_k_e': 10.0, 'EX_fe3_e': 10.0, 'EX_so4_e': 10.0, 'EX_fru_e': 3.0, 'EX_nh4_e': 10.0, 'EX_na_e': 10.0, 'EX_o2_e': 0, 'EX_mobd_e': 10.0, 'EX_h2o_e': 1000.0, 'EX_h_e': 100.0, 'EX_glc_e': 1000.0}
The current theoretical maximum biomass productivity: 0.0 h
----------------
The current maximum theoretical productivity for making PHB: 1.636363636363643 mmol/gDW*h
----------------
Maximum theoretical yield = 0.5454545454545476 mmol phb/mmol Fructose
{'EX_fe2_e': 10.0, 'EX_mg2_e': 10.0, 'EX_pi_e': 100.0, 'EX_cobalt2_e': 10.0, 'EX_cl_e': 10.0, 'EX_k_e': 10.0, 'EX_fe3_e': 10.0, 'EX_so4_e': 10.0, 'EX_fru_e': 3.0, 'EX_nh4_e': 10.0, 'EX_na_e': 10.0, 'EX_mobd_e': 10.0, 'EX_h2o_e': 1000.0, 'EX_h_e': 100.0, 'EX_glc_e': 1000.0, 'EX_o2_e': 10}


In [19]:
model.metabolites.get_by_id('phb_e')

Metabolite identifier,phb_e
Name,Poly-beta-hydroxybutyrate
Memory address,0x07ff96ae3ad90
Formula,None
Compartment,e
In 2 reaction(s),"EX_pbhb_e, PHBt"


In [20]:
medium['EX_succ_e']
medium['EX_fum_e']
medium['EX_pyr_e']

KeyError: 'EX_succ_e'